## Problem2

In [1]:
from os.path import exists
from google.colab import drive
import os
drive.mount('/content/dirve/', force_remount = True)
# change to your own directory
os.chdir("dirve/My Drive/Colab Notebooks/IFT6135/hw3/")
#!ls -al
from IPython.display import Image

Mounted at /content/dirve/


In [2]:
import numpy as np
import torch.nn as nn
import torch
import random
import math
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F

cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
print(cuda)

True


In [0]:
data_dir = "data/"
train_data = np.loadtxt(data_dir + "binarized_mnist_train.amat")
valid_data = np.loadtxt(data_dir + "binarized_mnist_valid.amat")
test_data = np.loadtxt(data_dir + "binarized_mnist_test.amat")

In [4]:
"""
for i, train_batch in enumerate(train_loader):
    print(train_batch.shape)
    for j in range(8):
        plt.subplot(1, 8, j+1)
        plt.imshow(train_batch[j].view(28,28).cpu().numpy())
    break
"""

'\nfor i, train_batch in enumerate(train_loader):\n    print(train_batch.shape)\n    for j in range(8):\n        plt.subplot(1, 8, j+1)\n        plt.imshow(train_batch[j].view(28,28).cpu().numpy())\n    break\n'

In [0]:
class Encoder(nn.Module):
    def __init__(self, z_dim = 100):
        super(Encoder, self).__init__()
        self.z_dim = z_dim
        self.encode = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = (3,3)),     # 28 -> 26
            nn.ELU(),
            nn.AvgPool2d(kernel_size = 2, stride = 2), # 26 -> 13
            nn.Conv2d(32, 64, kernel_size = (3,3)),    # 13 -> 11
            nn.ELU(),
            nn.AvgPool2d(kernel_size = 2, stride = 2), # 11 -> 5
            nn.Conv2d(64, 256, kernel_size = (5,5)),   # 5  -> 1
            nn.ELU(),
        )
        self.linear1 = nn.Linear(in_features = 256, out_features = 2*z_dim)
        
        
    def forward(self, x):                          # [batch_size, 1, 28, 28]
        q = self.encode(x)                         # [batch_size, 256, 1, 1]
        q = self.linear1(q.view(q.size(0), -1))    # [batch_size, 200]
        mu, log_var = q[:,:z_dim], q[:, z_dim: ]   # [batch_size, 100] 
        return mu, log_var
    
    
class Decoder(nn.Module):
    def __init__(self, z_dim = 100):
        super(Decoder, self).__init__()
        self.z_dim = z_dim
        self.linear2 = nn.Linear(in_features = z_dim, out_features = 256)
        self.decode = nn.Sequential(
            nn.ELU(),
            nn.Conv2d(256, 64, kernel_size=5, padding=4), 
            nn.ELU(),
            nn.UpsamplingBilinear2d(scale_factor=2),#mode='bilinear'
            nn.Conv2d(64, 32, kernel_size=3, padding=2),
            nn.ELU(),
            nn.UpsamplingBilinear2d(scale_factor=2),#mode='bilinear'
            nn.Conv2d(32, 16, kernel_size=3, padding=2),
            nn.ELU(),
            nn.Conv2d(16, 1, kernel_size=3, padding=2)
        )
        
        
    def forward(self, z):
        x_ = self.linear2(z).view(z.size(0),-1, 1,1) # [batch_size, 256, 1, 1]
        x_ = self.decode(x_)                       #[batch_size, 1, 28, 28] 
        return x_
        

class VAE(nn.Module):
    def __init__(self, z_dim = 100):
        super(VAE, self).__init__()
        self.encoder = Encoder(z_dim = z_dim)
        self.decoder = Decoder(z_dim = z_dim)
        
    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + torch.mul(eps, std)
    
    def forward(self, x):
        mu, log_var = self.encoder(x)
        z = self.reparameterize(mu, log_var)
        x_ = self.decoder(z)
        return x_, mu, log_var
        
    
def compute_elbo(recon_x, x, mu, log_var):    
    # compute log p_{\theta} (x)
    recon_x = recon_x.view(recon_x.size(0), -1)
    x = x.view(x.size(0), -1)
    
    # convert to the probabilities of each pixel to be 1, [batch_size, 1]
    output_px = torch.sigmoid(recon_x) 
    
    # compute the log probabilities of 
    # each reconstructed pixel value = original binary value
    # this is the binary cross entropy
    log_px = torch.log(output_px * x + (1- output_px) * (1-x))
    
    # log prob for a reconstructed image is original image x, [batch_size, 1]
    log_px = torch.sum(log_px, dim = 1, keepdim = True)
    log_px = torch.mean(log_px) # mean over batch_size samples
    # BCE = -1 * log_px         # relationship between BCE and lop_px
    
    # [batch_size, 1]
    KLD = 0.5 * torch.sum((-1 - log_var + torch.pow(mu, 2) + torch.exp(log_var)), 
                          dim = 1, keepdim = True)
    KLD = torch.mean(KLD) # mean over batch_size samples
    # mean elbo over batch_size samples, zero dimension
    elbo = log_px - KLD
    return elbo, -log_px, KLD
    
    
def compute_loss(recon_x, x, mu, log_var):
    batch_size = recon_x.size(0) # 
    # keep data and dimension, [batch_size, 1, 28, 28]
    BCE = F.binary_cross_entropy_with_logits(recon_x, x, reduction="none") 
    
    # sum all losses(of pixels) in one image, [batch_size, 1]
    BCE = torch.sum(BCE.view(batch_size, -1), dim = 1, keepdim = True)
    
    # compute average BCE over batch_size, zero dimension, []
    BCE = torch.mean(BCE) 
    
    # [batch_size, 1]
    KLD = 0.5 * torch.sum((-1 - log_var + torch.pow(mu, 2) + torch.exp(log_var)), 
                          dim = 1, keepdim = True)
    # average over batch_size, zero dimensiona, []
    KLD = torch.mean(KLD)
    return BCE + KLD, BCE, KLD


def data_loader(data, batch_size = 64):
    """
    params
        data: np.array, [sample_size, 784]
    """
    image_data = torch.unsqueeze(Tensor(data.reshape(-1, 28, 28)), 1)
    return DataLoader(image_data, batch_size = batch_size, shuffle = True)  

In [6]:
batch_size = 64
z_dim = 100
vae = VAE(z_dim = z_dim)
if cuda:
    vae.cuda()
    
optimizer = torch.optim.Adam(vae.parameters(), lr = 3e-4)

train_loader = data_loader(train_data, batch_size = batch_size)
valid_loader = data_loader(valid_data, batch_size = batch_size)

max_epochs = 20

for epoch in range(max_epochs):
    vae.train()
    for idx, x in enumerate(train_loader):
        recon_x, mu, log_var = vae(x)
        
        # both methods work
        #loss, bce, kld = compute_loss(recon_x, x, mu, log_var)
        elbo, bce, kld = compute_elbo(recon_x, x, mu, log_var)
        loss = -1 * elbo
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (idx+1) % 100 == 0:
            print ("Epoch[{}/{}], Step [{:5}], loss: {:.4f} BCE {:.4f}, KLD: {:.4f}"
                   .format(epoch+1, max_epochs, idx+1, 
                           -elbo.item(), bce.item(), kld.item()))
     
    vae.eval()
    val_elbo, val_loss, val_size = 0.0, 0.0, 0
    #val_size = len(valid_loader)
    for idx, x in enumerate(valid_loader):
        batch_size = x.shape[0]
        recon_x, mu, log_var = vae(x)
        loss, _, _ = compute_loss(recon_x, x, mu, log_var)
        elbo, _, _ = compute_elbo(recon_x, x, mu, log_var)
        val_elbo += elbo.item() * batch_size
        val_loss += loss.item() * batch_size
        val_size += batch_size
        
    val_loss /= val_size
    val_elbo /= val_size
    print("== Epoch[{}/{}]: elbo on valid set: {:.4f} {:.4f} ==\n"
          .format(epoch+1, max_epochs, -val_loss, val_elbo))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


Epoch[1/20], Step [  100], loss: 210.8652 BCE 202.3522, KLD: 8.5130
Epoch[1/20], Step [  200], loss: 181.9428 BCE 166.9125, KLD: 15.0303
Epoch[1/20], Step [  300], loss: 170.2816 BCE 151.8622, KLD: 18.4194
Epoch[1/20], Step [  400], loss: 163.6502 BCE 141.4412, KLD: 22.2089
Epoch[1/20], Step [  500], loss: 160.3189 BCE 138.5225, KLD: 21.7965
Epoch[1/20], Step [  600], loss: 149.8669 BCE 128.2368, KLD: 21.6301
Epoch[1/20], Step [  700], loss: 134.9668 BCE 114.4716, KLD: 20.4952
== Epoch[1/20]: elbo on valid set: -137.6708 -137.6708 ==

Epoch[2/20], Step [  100], loss: 131.4791 BCE 109.4372, KLD: 22.0419
Epoch[2/20], Step [  200], loss: 133.1040 BCE 110.7701, KLD: 22.3340
Epoch[2/20], Step [  300], loss: 127.1529 BCE 104.3317, KLD: 22.8212
Epoch[2/20], Step [  400], loss: 126.5841 BCE 103.2308, KLD: 23.3533
Epoch[2/20], Step [  500], loss: 115.7252 BCE 93.9899, KLD: 21.7354
Epoch[2/20], Step [  600], loss: 118.8657 BCE 96.5997, KLD: 22.2660
Epoch[2/20], Step [  700], loss: 114.1020 BCE 9

In [0]:
from torch.distributions.normal import Normal

M = 50 #batch_size
D, K, L = 784, 200, 100
#在下面的方法中 z,x 等小写字母表示 batch_size的张量，用z_i, x_i表示代表一个样本的向量。
#在计算x_i相关的概率或者log概率时， x_i都是一个784维的向量，
#每一个像素点之间的值相互独立，因此会发生784个值相乘或相加的情况。不要把x_i仅看成是一个标量

# method 1 YingXiao
def sampling(mu, log_var, k = K):
    """根据VAE的encoder，计算其对于单个x_i的输出(100维的mu， 100维的log_var)
    根据输出得到的100个对应的mu和log_var每次采样生成一个100维的向量z，共采样K次
    得到一个[K, L]维的tensor。这个sampling应该不需要加入噪声
    params
        mu: [1, L]
        log_var: [1, L]
        k: 要得到的采样的向量的数目
    returns
        z: [K, L]
    """
    sigma = torch.exp(0.5 * log_var)
    m = Normal(mu, sigma)
    z = torch.squeeze(m.sample(torch.Size([k])), 1)
    return z


# method 2 YingXiao
def log_normal_densities(z, mu, log_var):
    """density function of a normal distribution represented by mu, log_var
    对于标准正态分布，mu = Tensor([0, 0, ..., 0]), log_var = [0, 0, 0,... 0] ?
    该方法会两次用到，一次是计算标准正态分布，另一次使计算encoder得到的分布。
    params
        z: a tensor, shape [K, L]
        mu, [1, L]
        log_var, [1, L]
    returns
        log density value of z, tensor, [K, L]
    """
    log_prob_density = None
    sigma = torch.exp(0.5 * log_var)
    m = Normal(mu, sigma)
    log_probs = m.log_prob(z)
    # TODO compute the prob_density of z
    return torch.sum(log_probs, dim = 1, keepdim = True) # [K, 1]


# method 3 Marc
def log_probs_reconstruct(decoder, x_i, z):
    """根据提供的z, 将它送入decoder，得到重建的K个重建组成的 x_ , 对于其中的每一个x_i_,
    计算它是原来的x_i的log概率值
    params
        decoder: 
        x_i: [1, 784]
        z: [K, L] 来自于sampling得到的
    return
        log_p_theta: [K, 1]
    """
    k = z.size(0)
    recon_xs_i = decoder(z).view(k, -1)         # flatten [K, 784]
    recon_xs_i = torch.sigmoid(recon_xs_i)      # converto probabilities
    x_i = x_i.view(1, -1)
    xs_i = x_i.repeat(k, 1)                     # copy original x to K times
    # compare all 784 
    log_p_theta = torch.log(recon_xs_i * xs_i + (1- recon_xs_i) * (1-xs_i))
    return torch.sum(log_p_theta, dim = 1, keepdim = True)  # [K, 1]


# method 4 Marc
def generate_Z(encoder, x, k = K):
    """根据给定的x，使用sampling方法来生成Z
    params 
        encoder: 
        x: [M, D]
    returns
        Z: [M, K, L]
    """
    M = x.size(0)
    x = x.view(M, 1, 28, 28)
    mu, log_var = encoder(x)               # [batch_size, L]
    L = mu.size(1)
    Z = sampling(mu, log_var, k)
    Z = torch.transpose(Z, 0, 1)
    return Z
    
    
# method 5 Marc
def evaluate_log_likelihood(model, x, Z):
    """
    params
        model
        x: Tensor [M, D]
        Z: Tensor [M, K, L], given, usually based on x. so z can be generated
           by using the method sampling(mu, log_var, k) where mu, and log_var
           is generated by a x_i
    """
    encoder = model.encoder
    decoder = model.decoder
    mini_batch_size = x.size(0) # should be M
    x = x.view(mini_batch_size, -1) # [M, D]
    K = Z.size(1)          
    log_p = [0 for i in range(mini_batch_size)] #存放最终结果
    for i in range(mini_batch_size):
        x_i = x[i, :]           # [M, D]
        z = Z[i, :, :]          # [K, L]
        # 调用上面的方法
        mu, log_var = encoder(x_i.view(1, 1, 28, 28))    # [1, L], [1, L]
        mu0, log_var0 = torch.zeros(1, L), torch.zeros(1, L)
        if cuda:
            mu0 = mu0.cuda()
            log_var0 = log_var0.cuda()
        log_p_z = log_normal_densities(z, mu0, log_var0) # [K, 1] change 0 to Tensor 公式分子右侧
        log_q_z = log_normal_densities(z, mu, log_var) # [K, 1] 公式分母
        log_p_theta_x_i = log_probs_reconstruct(decoder, x_i, z) # [K, 1] 公式分子左侧
        
        # LogSumExp trick for underflow
        log_p_for_sum = log_p_theta_x_i + log_p_z - log_q_z      # [K, 1]
        max_log_p = torch.max(log_p_for_sum)
        log_p_for_sum1 = log_p_for_sum - max_log_p
        log_1k = torch.log(torch.tensor([1./K]))
        log_p_xi = log_1k + max_log_p + torch.log(torch.sum(torch.exp(log_p_for_sum1)))
        log_p[i] = log_p_xi.item()
    
    return log_p

In [0]:
encoder = vae.encoder
decoder = vae.decoder


In [0]:
# test sampling method
test_loader = data_loader(test_data, batch_size = 2)
x, x_i, mu, log_var = None, None, None, None

for batch_x in test_loader:
    x = batch_x
    print(x.shape)
    x_i = x[0,:,:]              
    print(x_i.shape)
    #mu, log_var = encoder(torch.unsqueeze(x_i,0))
    mu, log_var = encoder(x) # [batch_size, 1, 28, 28]
    print(mu.shape, log_var.shape)
    break
    
z = sampling(mu, log_var, k = int(100))# set k to very large number to check the mean 
                                 # and std of sampled data are the same of 
                                 # mu, sigma given.
N = 10 # only view first N datas
print("z.shape:", z.shape)
print(mu[:, :N])
print(torch.exp(0.5*log_var)[:, :N])
print(z[:, :N])
print(torch.mean(z, dim = 0, keepdim=True)[:, :N])
print(torch.std(z, dim = 0, keepdim = True)[:, :N])

In [18]:
print(x.shape)
Z = generate_Z(encoder, x, k = 200)
print(Z.shape)

torch.Size([2, 1, 28, 28])
torch.Size([2, 200, 100])


In [21]:
print(x_i.shape)


torch.Size([1, 28, 28])
torch.Size([100, 2, 100])


In [40]:
test_loader = data_loader(test_data[:2000,:], batch_size = 100)
model = vae
log_px = []

for idx, x in enumerate(test_loader):
    Z = generate_Z(model.encoder, x, k = 200)
    log_px.extend(evaluate_log_likelihood(model, x, Z))
    if idx % 10 == 0:
        print(idx)
    

mean_log_px = 0.0
for i in range(len(log_px)):
    mean_log_px += log_px[i]
    
mean_log_px /= len(log_px)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


0
10


In [41]:
print(mean_log_px)
print(log_px)

-72.74199173164368
[-100.45048522949219, -84.26823425292969, -91.2874984741211, -46.842281341552734, -188.6754913330078, -111.27562713623047, -114.24714660644531, -123.88329315185547, -45.454490661621094, -38.44133758544922, -39.660186767578125, -98.65153503417969, -103.92974853515625, -112.67644500732422, -42.04922103881836, -44.20904541015625, -36.917171478271484, -96.21465301513672, -94.85987091064453, -44.15841293334961, -102.69933319091797, -100.11200714111328, -102.05644226074219, -47.64393615722656, -79.23892211914062, -36.569854736328125, -106.16093444824219, -92.88619995117188, -87.11078643798828, -42.22732925415039, -111.6121826171875, -102.95740509033203, -109.14441680908203, -48.03965759277344, -103.39767456054688, -61.18047332763672, -97.57623291015625, -92.1852035522461, -100.83641052246094, -116.36680603027344, -45.481815338134766, -49.147422790527344, -41.53792190551758, -124.00205993652344, -91.871826171875, -92.16812896728516, -88.47905731201172, -96.36019897460938, -